# MEIER AEI Processing

This notebook processes the MEIER AEI (Area Equipped for Irrigation) dataset for Africa. It:
1. Loads the MEIER AEI raster data
2. Crops it to the Africa region
3. Converts it to vector format
4. Reprojects to Web Mercator (EPSG:3857)
5. Clips to different aridity layers
6. Adds country information
7. Saves the results as shapefiles

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import shape, box
from shapely.ops import unary_union
from tqdm import tqdm
from rasterio.plot import show
from rasterio.features import shapes
from rasterio.mask import mask
import rasterio
import numpy as np
import utility
import spatial_utility

## Load and Process MEIER AEI Raster Data

In [ ]:
try:
    # Load configuration
    config = utility.load_config()
    
    # Load and read the ASC file
    asc_file_path = utility.resolve_path(config['Africa_MEIER_AEI_2015_asc_path'])
    
    with rasterio.open(asc_file_path) as src:
        asc_data = src.read(1)  # Read the first band
        asc_meta = src.meta
        print("ASC metadata:", asc_meta)
        show(src, title="Original MEIER ASC Data")
    
    print(f"Loaded MEIER AEI raster with shape {asc_data.shape}")
except Exception as e:
    print(f"Error loading MEIER AEI raster: {e}")

## Crop to Africa Bounding Box

In [ ]:
try:
    # Define the bounding box for Africa
    africa_bbox = (-18.0, -35.0, 52.0, 38.0)  # (min_lon, min_lat, max_lon, max_lat)
    
    # Create a bounding box geometry
    bbox_geom = box(*africa_bbox)
    geo = gpd.GeoDataFrame({'geometry': [bbox_geom]}, index=[0], crs='EPSG:4326')
    
    # Reproject the bounding box to the ASC file CRS if necessary
    if asc_meta['crs'] != geo.crs:
        geo = geo.to_crs(crs=asc_meta['crs'])
    
    # Crop the ASC data using the bounding box
    with rasterio.open(asc_file_path) as src:
        out_image, out_transform = mask(src, [bbox_geom], crop=True)
        out_meta = src.meta.copy()
        out_meta.update({
            "driver": "GTiff",
            "height": out_image.shape[1],
            "width": out_image.shape[2],
            "transform": out_transform
        })
    
    # Save the cropped ASC data to a new file
    cropped_asc_path = utility.resolve_path(config['AEI_MEIER_2015_cropped_tif_path'])
    with rasterio.open(cropped_asc_path, 'w', **out_meta) as dest:
        dest.write(out_image)
    
    print(f"Cropped MEIER AEI raster saved to {cropped_asc_path}")
except Exception as e:
    print(f"Error cropping MEIER AEI raster: {e}")

## Convert Raster to Vector

In [ ]:
def raster_to_geodataframe(raster_path, crs):
    """Convert a raster to a GeoDataFrame."""
    with rasterio.open(raster_path) as src:
        image = src.read(1)  # Read the first band
        mask = image != src.nodata
        results = (
            {'properties': {'raster_val': v}, 'geometry': shape(s)}
            for i, (s, v) in enumerate(shapes(image, mask=mask, transform=src.transform))
        )
        geoms = list(results)
        gdf = gpd.GeoDataFrame.from_features(geoms)
        gdf.crs = crs
    return gdf

try:
    # Convert the cropped raster to a GeoDataFrame
    gdf_cropped_asc = raster_to_geodataframe(cropped_asc_path, asc_meta['crs'])
    print(f"Converted raster to GeoDataFrame with {len(gdf_cropped_asc)} features")
    print("Sample data:")
    print(gdf_cropped_asc.head())
    
    # Plot the cropped GeoDataFrame
    fig, ax = plt.subplots(figsize=(10, 10))
    gdf_cropped_asc.boundary.plot(ax=ax, linewidth=1, edgecolor='blue', alpha=0.5, label='Cropped MEIER ASC Data')
    plt.title('Cropped MEIER ASC Data')
    plt.legend()
    plt.show()
except Exception as e:
    print(f"Error converting raster to GeoDataFrame: {e}")

## Reproject to Web Mercator

In [ ]:
try:
    # Reproject the cropped GeoDataFrame to Web Mercator (EPSG:3857)
    gdf_cropped_asc_reprojected = gdf_cropped_asc.to_crs(epsg=3857)
    
    # Print CRS and bounding box after reprojection
    print("Reprojected CRS:", gdf_cropped_asc_reprojected.crs)
    print("Reprojected Bounding Box:", gdf_cropped_asc_reprojected.total_bounds)
    
    # Plot the reprojected GeoDataFrame
    fig, ax = plt.subplots(figsize=(10, 10))
    gdf_cropped_asc_reprojected.boundary.plot(ax=ax, linewidth=1, edgecolor='red', alpha=0.5, label='Reprojected MEIER ASC Data')
    plt.title('Reprojected MEIER ASC Data')
    plt.legend()
    plt.show()
    
    # Save the reprojected GeoDataFrame as a GeoPackage
    geopackage_path = utility.resolve_path(config['AEI_MEIER_2015_reproj_gpkg_path'])
    gdf_cropped_asc_reprojected.to_file(geopackage_path, driver="GPKG")
    print(f"Reprojected GeoDataFrame saved to {geopackage_path}")
except Exception as e:
    print(f"Error reprojecting GeoDataFrame: {e}")

## Process by Aridity Layers

In [ ]:
try:
    # Process the MEIER AEI data by aridity layers using the utility function
    results = spatial_utility.process_aei_by_aridity(dataset_type="meier")
    
    # Print summary of results
    for layer, gdf in results.items():
        print(f"Layer {layer}: {len(gdf)} features")
except Exception as e:
    print(f"Error processing MEIER AEI by aridity layers: {e}")